## 🔎 Agent 可观测性 - 日志、追踪与指标(LangChain可本地运行版)

欢迎来到 Kaggle 5-day Agents 课程的第 4 天！

在第 3 天，你学习了会话和记忆管理（Session and Memory）。

今天，你将学习：
1.  如何为你构建的 Agent 添加**可观测性 (Observability)**。
2.  如何评估 Agent 是否按预期工作。

本 Notebook 将重点关注第一部分 - **Agent 可观测性**！

### 什么是 Agent 可观测性？

🚨 **挑战**：与传统的软件不同，AI Agent 的失败往往比较“神秘”。例如：

*   用户：“查找量子计算的论文”
*   Agent：“我无法帮助处理该请求。”
*   你：😭 为什么？？是 Prompt 的问题？缺少工具？还是 API 错误？

💡 **解决方案**：Agent 可观测性让你能够完全洞察 Agent 的决策过程。你可以清楚地看到发送给 LLM 的 Prompt、可用的工具、模型的回复以及失败发生的位置。

### Agent 可观测性的基础支柱

*   **日志 (Logs)**：记录单个事件，告诉你特定时刻发生了什么。
*   **追踪 (Traces)**：将日志连接成一个完整的故事，展示整个步骤序列，解释最终结果是如何产生的。
*   **指标 (Metrics)**：总结性数字（如平均值、错误率），反映 Agent 的整体表现。

在本 Notebook 中，你将：

1.  ✅ 设置日志配置
2.  ✅ 创建一个“由于逻辑错误而失效”的 Agent，并使用日志找出原因
3.  ✅ 在 LangChain 中实现生产级日志记录 (Callbacks)
4.  ✅ 学习如何使用自定义 Handler 进行调试
---
**原版**: Google ADK Team (Kaggle 5-Day Agent Course)  
**改写**: LangChain 实现
> **ℹ️ 声明**: 本教程改编自 Google [5-Day AI Agents Course](https://www.kaggle.com/learn-guide/5-day-agents)，由 LangChain 重写以适配本地运行。内容仅供学习研究。

## ⚙️ Section 1: Setup (环境设置)

### 1.1 切换内核
- 点击右上角的 Kernel (内核) 选择区域。
- 在列表中选择 Python (5-DAY-AGENTS)。

> 关于如何初始化环境和创建内核，请查看 [README 如何运行](../README.md#如何运行)

### 1.2 安装依赖

In [ ]:
# 优选 uv 包管理器
!uv pip install langchain langchain-openai langchain-community duckduckgo-search tavily-python python-dotenv


### 1.3 导入依赖库

In [1]:
import os
import json
import warnings
from dotenv import load_dotenv, find_dotenv

# 忽略废弃警告
from langchain_core._api.deprecation import LangChainDeprecationWarning
warnings.simplefilter('ignore', category=LangChainDeprecationWarning)

# 过滤 Pydantic 相关警告
try:
    from pydantic.warnings import PydanticDeprecatedSince20
    warnings.filterwarnings('ignore', category=PydanticDeprecatedSince20)
except ImportError:
    pass
warnings.filterwarnings('ignore', message='.*PydanticDeprecatedSince211.*')
warnings.filterwarnings('ignore', message='.*Accessing the \'model_fields\' attribute.*')
warnings.filterwarnings('ignore')

print("✅ 依赖库加载完成")

✅ 依赖库加载完成


### 1.4 配置 API Key

In [2]:
# 强制重新加载环境变量，确保获取最新 Key
load_dotenv(find_dotenv(), override=True)

def configure_api_keys():
    """加载并验证 API 配置"""
    model_name = os.environ.get('MODEL_NAME', 'gpt-3.5-turbo')
    api_key = os.environ.get('OPENAI_API_KEY')
    base_url = os.environ.get('OPENAI_API_BASE')
    tavily_key = os.environ.get('TAVILY_API_KEY')

    if not api_key:
        print('❌ 错误: 未在 .env 文件中找到 OPENAI_API_KEY')
        return None
    
    # 打印部分 Key 以验证
    masked_key = f'{api_key[:8]}...' if api_key else 'None'
    print(f'✅ API Key 已加载: {masked_key}')
    print(f'✅ 使用模型: {model_name}')
    if base_url:
        print(f'✅ API Base URL: {base_url}')
        
    # Explicitly check Tavily Key
    if tavily_key:
         print(f'✅ Tavily Key 已加载: {tavily_key[:8]}...')
    else:
         print('⚠️ 未检测到 TAVILY_API_KEY，将使用 DuckDuckGo 作为搜索工具。')
    
    return {
        'model_name': model_name,
        'api_key': api_key,        'base_url': base_url,
        'tavily_key': tavily_key,
        'extra_body': json.loads(os.environ.get('EXTRA_BODY', '{}'))
    }

config = configure_api_keys()

✅ API Key 已加载: ms-04d57...
✅ 使用模型: Qwen/Qwen3-235B-A22B-Instruct-2507
✅ API Base URL: https://api-inference.modelscope.cn/v1
✅ Tavily Key 已加载: tvly-dev...


### 1.5 设置基础日志

在 Python 中，我们可以使用标准的 `logging` 模块。LangChain 也集成了这一点。让我们先清理一下旧的日志文件。

In [3]:
import logging
import os

# 清理旧日志
log_filename = 'agent_debug.log'
if os.path.exists(log_filename):
    os.remove(log_filename)
    print(f'🧹 已清理旧日志文件: {log_filename}')

# 配置基础日志
logging.basicConfig(
    filename=log_filename,
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

print('✅ 日志已配置')

✅ 日志已配置


## 🐞 第二部分：动手调试 (Hands-On Debugging)

### 2.1 创建一个“论文查找助手” Agent

我们的目标：构建一个 Agent，帮助用户查找特定主题的学术论文并统计数量。

但首先，为了练习调试，我们故意创建一个**有缺陷**的版本！

我们将定义两个工具：
1.  `search_papers`: 使用 Tavily 搜索论文。
2.  `count_papers`: 统计论文数量。

👉 **注意**：仔细观察 `count_papers` 的定义，它是 bug 的来源！

In [22]:
from langchain.tools import tool
# 使用与 Day 1a 一致的社区版导入
from langchain_community.tools import DuckDuckGoSearchRun, TavilySearchResults
from langchain_openai import ChatOpenAI
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate
import os

# 1. 定义搜索工具
ddg_search = DuckDuckGoSearchRun()

@tool
def search_papers(query: str) -> str:
    """搜索关于给定主题的研究论文信息。"""
    
    # 优先尝试 Tavily (如果配置了 Key)
    # 这与 Day 1a 的实现方式保持一致
    if os.environ.get('TAVILY_API_KEY'):
        try:
            tavily = TavilySearchResults(max_results=2)
            # Tavily 返回的是 List[Dict]，转为字符串给 LLM
            results = tavily.invoke(f'research papers about {query}')
            return str(results)
        except Exception as e:
            print(f'⚠️ Tavily 搜索不可用 ({e})，切换到 DuckDuckGo...')

    # 备用方案：DuckDuckGo
    try:
        result = ddg_search.run(f'research papers about {query}')
        # 简单检查结果有效性
        if not result or len(result) < 50 or "No good DuckDuckGo Search Result" in result: 
             raise Exception('Search result invalid or too short')
        return result
    except Exception as e:
        # 最后防线：模拟数据
        print(f'⚠️ 搜索工具受限 ({e})，使用模拟数据演示。')
        return """
        Here are some significant research papers on the topic:
        1. "Quantum Supremacy Using a Programmable Superconducting Processor" - demonstrating quantum advantage.
        2. "A Variational Eigenvalue Solver on a Photonic Quantum Processor" - advances in hybrid algorithms.
        3. "Quantum Error Correction for Quantum Memories" - key steps towards fault-tolerant computing.
        4. "Scalable Quantum Computing with Trapped Ions" - exploring ion trap architectures.
        """

# 2. 定义计数工具 - 故意保留 Bug
# ❌ BUG: LLM 可能会传入一个长字符串而不是列表
@tool
def count_papers(papers: str) -> int:
    """
    计算列表中的论文数量。
    Args:
        papers: 包含论文标题的字符串。
    Returns:
        论文的数量。
    """
    return len(papers)

tools = [search_papers, count_papers]

# 3. 创建 Agent
llm = ChatOpenAI(
    model=config['model_name'],
    api_key=config['api_key'],
    base_url=config['base_url'],
    temperature=0,
    model_kwargs={'extra_body': config['extra_body']}
)

prompt = ChatPromptTemplate.from_messages([
    ('system', """你的任务是查找研究论文并统计它们。
    你必须遵循以下步骤：
    1) 使用 'search_papers' 工具查找关于用户提供主题的论文。
    2) 将找到的所有论文信息合并成一个长字符串，并直接传递给 'count_papers' 工具得到返回值 'paper_count'。
    3) 返回:
    论文列表
    论文总数: 'paper_count' (重要：必须严格直接使用 count_papers 工具返回的数值作为总数，绝对禁止自己手动计数)
    """),
    ('human', '{input}'),
    ('placeholder', '{agent_scratchpad}'),
])

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)

print('✅ 包含 Bug 的 Agent 已创建')

✅ 包含 Bug 的 Agent 已创建


### 2.2 运行 Agent

让我们运行 Agent。注意观察结果中的数字。

In [23]:
response = agent_executor.invoke({'input': '查找关于量子计算的最新论文'})
print(f"Agent 回复: {response['output']}")

Agent 回复: 论文列表  
1. 《量子计算发展态势研究报告 (2025 年)》 - 来源：中国信息通信研究院、中移（苏州）软件技术有限公司和北京玻色量子科技有限公司  
2. 《Quantum Computing: Algorithms and Applications - A Literature Review》 - 作者：Rion, Abid Hossain 和 Any, Mashrufa Meghla，发布于 SSRN，2025年8月4日  

论文总数: 2457


🤔 **结果分析**：
你可能会看到类似这样的回复：“论文总数: 2457 。”（或者其他很大的数字）。

显然，一次搜索不可能返回几百篇论文。这个数字太大了，看起来像是字符数，而不是篇数。

**我们如何调试这个问题？**

### 2.3 使用 LangChain 的可观测性工具

在 LangChain 中，我们可以通过设置 `verbose=True` 或者使用 **Callbacks** 来查看 Agent 的思考过程（Trace）。这相当于原教程中的 `log_level DEBUG`。

让我们将 `verbose=True` 并使用 `StdOutCallbackHandler` 来重新运行。

In [24]:
from langchain_core.tracers import ConsoleCallbackHandler

# 重新创建 Executor，开启 verbose
debug_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# 运行并观察输出
print('🔍 开始调试运行...\n')
debug_executor.invoke({'input': '查找关于量子计算的最新论文'})

🔍 开始调试运行...



> Entering new AgentExecutor chain...

Invoking: `search_papers` with `{'query': '量子计算'}`


[{'url': 'http://221.179.172.81/images/20250930/97631759200863191.pdf', 'content': '中国信息通信研究院技术与标准研究所 中移（苏州）软件技术有限公司 北京玻色量子科技有限公司 2025年9月 量子计算发展态势研究报告 (2025 年) 版权声明 本报告版权属于中国信息通信研究院、中移（苏州） 软件技术有限公司和北京玻色量子科技有限公司，并受法 律保护。转载、摘编或利用其它方式使用本报告文字或者 观点的，应注明“来源：中国信息通信研究院、中移（苏州） 软件技术有限公司和北京玻色量子科技有限公司”。违反上 述声明者，编者将追究其相关法律责任。 前 言 量子计算以量子比特为基本单位，基于量子叠加和量子纠缠等 特性实现高效并行运算，有望为解决复杂计算难题提供指数级加速， 成为推动未来算力跨域式发展重要方向，兼具重要科学价值和战略 意义。量子计算已成为全球科技强国投资布局的重点方向，各国纷 纷加大在前沿技术攻关、原型机研发、应用场景探索以及产业生态 构建等方面投入力度，量子计算国际竞争正日趋白热化。 量子计算领域正处于技术突破、应用探索和产业培育同步推进 的关键期。超导、离子阱、中性原子、光量子、硅半导体、拓扑等 技术路线创新成果不断涌现，原型机性能指标持续提升。量子计算 [...] 门和读取保真度分别达到99.90%、99.62%和99.18%10。2025 年，中 科大联合团队等基于“祖冲之三号”求解随机线路采样任务并获得比 超级计算机Frontier 高15 个数量级的求解速度11。日本理化学研究所 与富士通合作研发具有可扩展连接架构的256 比特超导量子计算机12。 除关键指标提升外，超导路线相关的科研成果也不断涌现。中科院 物理所联合团队实验验证用于超导量子处理器中高保真双量子比特 门的脉冲校准方案，简化校准过程的同时提高了磁通控制的精度和 稳定性13。哈佛大学联合团队开发微波-光学量子转导器，可实现光 学驱动的超导量子比特相干操控，光-微波转换效率达到1.18%14。浙 

{'input': '查找关于量子计算的最新论文',
 'output': '论文列表:\n1. 中国信息通信研究院技术与标准研究所 中移（苏州）软件技术有限公司 北京玻色量子科技有限公司 2025年9月 量子计算发展态势研究报告 (2025 年)\n   - URL: http://221.179.172.81/images/20250930/97631759200863191.pdf\n   - 内容摘要: 报告分析了量子计算的技术发展、应用探索和产业生态，涵盖超导、离子阱、中性原子、光量子等多种技术路线的最新进展，并讨论了其在金融、化工、生物、交通等领域的应用场景以及与人工智能的融合趋势。\n\n2. Quantum Computing: Algorithms and Applications - A Literature Review\n   - URL: https://papers.ssrn.com/sol3/papers.cfm?abstract_id=5380433\n   - 内容摘要: 这篇综述研究了量子算法与量子应用的交叉领域，强调了量子机器学习、量子科学计算和算法原语方面的许多进展，突出了量子系统如何重塑各个领域的计算环境。\n\n论文总数: 2632'}

### 2.4 分析日志

查看上面的绿色/彩色日志输出（Trace）：

1.  找到 ' Invoking: count_papers with '。
2.  你可能会看到 LLM 将论文标题合并成了一个长字符串（例如 `Paper A\nPaper B`），而不是传入一个列表。
3.  Agent 将这个拼接后的字符串传给了 `count_papers`。
4.  我们的 `count_papers` 函数执行了 `len(text)`，计算的是**字符数**（例如 100+），而不是**列表项数**（例如 2 或 3）。

🔎 **发现根本原因 (Root Cause)**：
工具定义的输入类型是 `str`，且逻辑没有处理字符串分割。LLM 忠实地遵循了指令，传入了一个包含多篇论文标题的字符串，但我们的代码逻辑是 `len(string)`，直接计算了字符长度。

In [25]:
from typing import List

# ✅ 修复版工具：明确接收 List[str] 或者在内部进行处理
# LangChain 的 LLM 通常能很好地处理 List[str] 类型提示

@tool
def count_papers_fixed(papers: List[str]) -> int:
    """
    计算列表中的论文数量。
    Args:
        papers: 论文标题的列表。
    Returns:
        论文的数量。
    """
    return len(papers)

# 更新工具列表
tools_fixed = [search_papers, count_papers_fixed]

# 重新创建 Agent
agent_fixed = create_tool_calling_agent(llm, tools_fixed, prompt)
fixed_executor = AgentExecutor(agent=agent_fixed, tools=tools_fixed, verbose=True)

print('✅ 修复后的 Agent 已创建')

# 再次运行
print('🚀 运行修复后的 Agent...\n')
response_fixed = fixed_executor.invoke({'input': '查找关于量子计算的最新论文'})
print(f"\nAgent 最终回复: {response_fixed['output']}")

✅ 修复后的 Agent 已创建
🚀 运行修复后的 Agent...



> Entering new AgentExecutor chain...

Invoking: `search_papers` with `{'query': '量子计算'}`


[{'url': 'http://221.179.172.81/images/20250930/97631759200863191.pdf', 'content': '中国信息通信研究院技术与标准研究所 中移（苏州）软件技术有限公司 北京玻色量子科技有限公司 2025年9月 量子计算发展态势研究报告 (2025 年) 版权声明 本报告版权属于中国信息通信研究院、中移（苏州） 软件技术有限公司和北京玻色量子科技有限公司，并受法 律保护。转载、摘编或利用其它方式使用本报告文字或者 观点的，应注明“来源：中国信息通信研究院、中移（苏州） 软件技术有限公司和北京玻色量子科技有限公司”。违反上 述声明者，编者将追究其相关法律责任。 前 言 量子计算以量子比特为基本单位，基于量子叠加和量子纠缠等 特性实现高效并行运算，有望为解决复杂计算难题提供指数级加速， 成为推动未来算力跨域式发展重要方向，兼具重要科学价值和战略 意义。量子计算已成为全球科技强国投资布局的重点方向，各国纷 纷加大在前沿技术攻关、原型机研发、应用场景探索以及产业生态 构建等方面投入力度，量子计算国际竞争正日趋白热化。 量子计算领域正处于技术突破、应用探索和产业培育同步推进 的关键期。超导、离子阱、中性原子、光量子、硅半导体、拓扑等 技术路线创新成果不断涌现，原型机性能指标持续提升。量子计算 [...] 门和读取保真度分别达到99.90%、99.62%和99.18%10。2025 年，中 科大联合团队等基于“祖冲之三号”求解随机线路采样任务并获得比 超级计算机Frontier 高15 个数量级的求解速度11。日本理化学研究所 与富士通合作研发具有可扩展连接架构的256 比特超导量子计算机12。 除关键指标提升外，超导路线相关的科研成果也不断涌现。中科院 物理所联合团队实验验证用于超导量子处理器中高保真双量子比特 门的脉冲校准方案，简化校准过程的同时提高了磁通控制的精度和 稳定性13。哈佛大学联合团队开发微波-光学量子转导器，可实现光 学驱动的超导量子比特相干

🎉 **调试成功**！现在你应该能看到一个合理的数字（例如 2、3 或 4），因为 LLM 现在正确地传递了一个列表给工具。

---

## 🧑‍💻 第三部分：生产环境的日志记录 (Logging in Production)

在开发阶段，使用 `print` 或 `verbose=True` 直接看控制台输出很方便。但在生产环境中，你需要自动化地记录这些信息。

在 LangChain 中，这通过 **Callbacks** 机制实现。你可以编写自定义的 `CallbackHandler` 来拦截 Agent 生命周期的各个阶段（如：开始运行、工具调用、LLM 开始生成、结束等）。

### 3.1 创建自定义日志 Handler

让我们创建一个类似于原教程中 `LoggingPlugin` 的 Handler，将关键事件记录到文件中。

In [26]:
from langchain_core.callbacks import BaseCallbackHandler
from langchain_core.agents import AgentAction, AgentFinish
from typing import Any, Dict, List, Union

class FileLoggingHandler(BaseCallbackHandler):
    """一个自定义的 Handler，将 Agent 的活动记录到文件中"""
    
    def __init__(self, filename):
        self.filename = filename
        
    def _log(self, message):
        with open(self.filename, 'a', encoding='utf-8') as f:
            f.write(message + '\n')

    def on_chain_start(self, serialized: Dict[str, Any], inputs: Dict[str, Any], **kwargs: Any) -> Any:
        self._log(f'[Chain Start] 输入: {inputs}')

    def on_tool_start(self, serialized: Dict[str, Any], input_str: str, **kwargs: Any) -> Any:
        self._log(f"[Tool Start] 调用工具: {serialized.get('name')} | 输入: {input_str}")

    def on_tool_end(self, output: str, **kwargs: Any) -> Any:
        self._log(f'[Tool End] 工具输出: {output[:100]}...') # 截断长输出

    def on_agent_action(self, action: AgentAction, **kwargs: Any) -> Any:
        self._log(f'[Agent Action] 决定执行: {action.tool} | 输入: {action.tool_input}')

    def on_agent_finish(self, finish: AgentFinish, **kwargs: Any) -> Any:
        self._log(f"[Agent Finish] 最终结果: {finish.return_values['output']}")

# 实例化 Handle
file_logger = FileLoggingHandler('production_agent.log')

print('✅ 自定义 FileLoggingHandler 已就绪')

✅ 自定义 FileLoggingHandler 已就绪


### 3.2 使用自定义 Handler 运行 Agent

现在我们将这个 Handler 传递给 Agent。注意，我们不再需要在控制台看到 `verbose` 输出，而是可以在后台默默记录。

In [27]:
# 关闭 verbose，但在 invoke 时传入 callbacks
prod_executor = AgentExecutor(agent=agent_fixed, tools=tools_fixed, verbose=False)

print('🚀 生产模式运行中 (无控制台详细输出)...')
prod_executor.invoke(
    {'input': '查找关于 AI Agent 可观测性的论文'},
    config={'callbacks': [file_logger]}  # 注入我们的 Logger
)
print('✅ 运行结束。')

🚀 生产模式运行中 (无控制台详细输出)...
✅ 运行结束。


### 3.3 检查生产日志

让我们读取生成的日志文件，看看记录了什么。

In [28]:
print(f'📄 读取日志文件 {file_logger.filename}:\n')
with open(file_logger.filename, 'r', encoding='utf-8') as f:
    print(f.read())

📄 读取日志文件 production_agent.log:

[Chain Start] 输入: {'input': '查找关于 AI Agent 可观测性的论文'}
[Chain Start] 输入: {'input': ''}
[Chain Start] 输入: {'input': ''}
[Chain Start] 输入: {'input': ''}
[Chain Start] 输入: {'input': ''}
[Chain Start] 输入: {'input': '查找关于 AI Agent 可观测性的论文', 'intermediate_steps': [], 'agent_scratchpad': []}
[Chain Start] 输入: content='' additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_4f404102d9b343d9948239', 'function': {'arguments': '{"query": "AI Agent 可观测性"}', 'name': 'search_papers'}, 'type': 'function'}]} response_metadata={'finish_reason': 'tool_calls', 'model_name': 'Qwen/Qwen3-235B-A22B-Instruct-2507'} id='run-99e0a5dd-ad02-4f87-9f23-e269fc8ef82e' tool_calls=[{'name': 'search_papers', 'args': {'query': 'AI Agent 可观测性'}, 'id': 'call_4f404102d9b343d9948239', 'type': 'tool_call'}] usage_metadata={'input_tokens': 0, 'output_tokens': 0, 'total_tokens': 0, 'input_token_details': {}, 'output_token_details': {}} tool_call_chunks=[{'name': 'search_papers', 'args': '{"qu

## 📊 总结

恭喜！你已经学习了如何为 AI Agent 添加可观测性。

1.  **调试 (Development)**：在开发过程中，使用 `verbose=True` 和 `ConsoleCallbackHandler` 来实时查看 Agent 的“思考过程”和“工具使用”，这有助于快速定位像类型不匹配这样的逻辑错误。
2.  **生产 (Production)**：在生产环境中，使用自定义的 **Callbacks** 将关键事件（Chain 开始、Tool 调用、Agent 结束）记录到文件或监控系统中，以便在不干扰用户体验的情况下进行后续分析。

### 关键学习点
*   **症状 -> 日志 -> 根因 -> 修复**：这是核心的调试循环。
*   **LangChain Callbacks**：是实现 Observability 的核心机制，等同于原教程中的 Plugin 系统。

现在，你的 Agent 不再是一个“黑盒”，你可以完全掌握它的每一次决策！🚀